In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib, random
from submission import clean_df

# loading data (predictors)
train = pd.read_csv("..\\data\\training_data\\PreFer_train_data.csv", low_memory = False) 
# loading the outcome
outcome = pd.read_csv("..\\data\\training_data\\PreFer_train_outcome.csv") 

In [2]:
str_train = train.select_dtypes(include=['object'])

bool_train = train.select_dtypes(include=['bool'])
cat_train = train.select_dtypes(include=['category'])

num_train = train.select_dtypes(exclude=['object'])

In [3]:
from category_encoders import *

cb = CatBoostEncoder(verbose=1)
cb.fit(train, outcome['new_child']);

enc_train = cb.transform(train)

In [6]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(enc_train)
ie_train = imp.transform(enc_train);


In [9]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

featsel = BorutaPy(estimator=rf, n_estimators='auto', verbose=2, random_state=1)
featsel.fit(ie_train, outcome['new_child']);

# check selected features - first 5 features are selected
featsel.support_

# check ranking of features
featsel.ranking_

# call transform() on X to filter it down to selected features
rel_train = featsel.transform(enc_train)

ValueError: Input y contains NaN.